## Dependencies

In [83]:
# Import dependencies
# NFL pbp data
import nfl_data_py as nfl

# Basics / visualizations
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

# Machine learning tools
import lightgbm as lgb
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from skopt import BayesSearchCV

# Stats Stuff
from scipy import stats
from scipy.stats import skew
import statsmodels.api as sm


# Turn off max columns for pandas DataFrame
pd.set_option('display.max_columns', None)

## Importing and Prepping Schedules

In [84]:
schedules_df = nfl.import_schedules([2024])

In [142]:
def get_upcoming_week(schedule):
    """
    Finds the nearest upcoming week where games have not yet been played.
    """
    # Filter out games where both home_score and away_score are NaN (future games)
    future_games = schedule[schedule['home_score'].isna() & schedule['away_score'].isna()]
    
    # Find the soonest week with future games
    upcoming_week = future_games['week'].min()
    
    return upcoming_week

In [143]:
get_upcoming_week(schedules_df)

4

## Importing PBP Data

In [144]:
# Select only the relevant columns
columns = ['game_id', 'passer_player_name', 'posteam', 'defteam', 'season', 'week', 'home_team', 'away_team', 'play_type', 'air_yards', 
           'yards_after_catch', 'epa', 'complete_pass', 'incomplete_pass', 'interception', 'qb_hit', 'sack', 'pass_touchdown',
           'passing_yards', 'cpoe', 'roof', 'surface']

# Loading in the NFL pbp data
data = nfl.import_pbp_data(range(2014,2025), columns, include_participation=False)

# nfl-data-py still loads other columns, so we again need to set our data equal to only the columns we want
data = data[columns]

# Drop all rows that are not a pass
data = data[data['play_type'] == 'pass']

# Drop the play type column
passer_data = data.drop(columns=['play_type'])

2014 done.
2015 done.
2016 done.
2017 done.
2018 done.
2019 done.
2020 done.
2021 done.
2022 done.
2023 done.
2024 done.
Downcasting floats.


## Passer DF Prep

In [180]:
# Group the data together by passer, week, season and aggregate
passer_df = passer_data.groupby(['game_id', 'passer_player_name', 'week', 'season'], as_index=False).agg(
    {'posteam' : 'first',
     'defteam' : 'first',
     'home_team' : 'first',
     'away_team' : 'first',
     'air_yards' : 'sum',
     'yards_after_catch' : 'sum',
     'epa' : 'sum',
     'complete_pass' : 'sum',
     'incomplete_pass' : 'sum',
     'interception' : 'sum',
     'qb_hit' : 'sum',
     'sack' : 'sum',
     'pass_touchdown' : 'sum',
     'passing_yards' : 'sum',
     'cpoe' : 'mean',
     'roof' : 'first',
     'surface' : 'first'
     }
)

# Create a new column that is completion percentage
passer_df['completion_percentage'] = passer_df['complete_pass'] / (passer_df['complete_pass'] + passer_df['incomplete_pass'])

# Create a new column that is the number of pass attempts
passer_df['pass_attempts'] = passer_df['complete_pass'] + passer_df['incomplete_pass']

# Drop the complete_pass and incomplete_pass columns
passer_df = passer_df.drop(columns=['complete_pass', 'incomplete_pass'])

# Create a new column that equals 1 if the passer is the home team and 0 if the passer is the away team
passer_df['home_flag'] = passer_df['home_team'] == passer_df['posteam']

# Drop the home_team and away_team columns
passer_df = passer_df.drop(columns=['home_team', 'away_team'])

# Reorder the columns
passer_df = passer_df[['game_id', 'passer_player_name', 'posteam', 'defteam', 'season', 'week', 'passing_yards', 'home_flag', 'completion_percentage', 'pass_attempts',
                       'air_yards',  'yards_after_catch', 'epa', 'interception', 'qb_hit', 'sack', 'pass_touchdown', 
                        'cpoe', 'roof', 'surface']]

## Defense DF Prep

In [181]:
# Select only the relevant columns
defense_columns = ['defteam', 'season', 'week', 'home_team', 'away_team', 'play_type', 'air_yards',
                   'yards_after_catch', 'epa', 'complete_pass', 'incomplete_pass', 'interception', 'qb_hit', 'sack', 'pass_touchdown',
                   'passing_yards', 'cpoe', 'roof', 'surface']


# nfl-data-py still loads other columns, so we again need to set our data equal to only the columns we want
defense_data = data[defense_columns]

# Drop the play type column
defense_data = defense_data.drop(columns=['play_type'])

# Group the data together by passer, week, season and aggregate
defense_df = defense_data.groupby(['defteam', 'week', 'season'], as_index=False).agg(
    {'home_team': 'first',
     'away_team': 'first',
     'air_yards': 'sum',
     'yards_after_catch': 'sum',
     'epa': 'sum',
     'complete_pass': 'sum',
     'incomplete_pass': 'sum',
     'interception': 'sum',
     'qb_hit': 'sum',
     'sack': 'sum',
     'pass_touchdown': 'sum',
     'passing_yards': 'sum',
     'cpoe': 'mean',
     'roof': 'first',
     'surface': 'first'
     }
)

# Create a new column that is completion percentage
defense_df['completion_percentage'] = defense_df['complete_pass'] / (defense_df['complete_pass'] + defense_df['incomplete_pass'])

# Create a new column that is the number of pass attempts
defense_df['pass_attempts'] = defense_df['complete_pass'] + defense_df['incomplete_pass']

# Drop the complete_pass and incomplete_pass columns
defense_df = defense_df.drop(columns=['complete_pass', 'incomplete_pass'])

# Create a new column that equals 1 if the defense is the home team and 0 if the defense is the away team
defense_df['home_flag'] = defense_df['home_team'] == defense_df['defteam']

# Drop the home_team and away_team columns
defense_df = defense_df.drop(columns=['home_team', 'away_team'])

# Reorder the columns
defense_df = defense_df[['defteam', 'season', 'week', 'home_flag', 'passing_yards', 'completion_percentage', 'pass_attempts',
                       'air_yards',  'yards_after_catch', 'epa', 'interception', 'qb_hit', 'sack', 'pass_touchdown', 
                       'cpoe', 'roof', 'surface']]

In [182]:
def filter_most_recent_weeks(df, weeks_needed=2):
    """
    Filters the dataframe to only include the most recent weeks needed for EWMA calculation.
    Automatically handles offense and defense data by checking for 'passer_player_name' or 'defteam'.
    
    :param df: The dataframe to filter (either passer_df or defense_df)
    :param weeks_needed: Number of recent weeks needed for EWMA calculation
    :return: Filtered dataframe with only the most recent weeks
    """
    # Dynamically detect whether to use passer_player_name or defteam
    group_column = 'passer_player_name' if 'passer_player_name' in df.columns else 'defteam'
    
    # Sort by the group column and 'week' in descending order
    df = df.sort_values([group_column, 'week'], ascending=[True, False])
    
    # Group by the detected column and get the most recent N weeks
    recent_weeks = df.groupby(group_column).head(weeks_needed)
    
    return recent_weeks

In [183]:
def calculate_passer_ewma(passer_df):
    """
    Calculates EWMA for the passer (offense) dataframe.
    """
    passer_df['completion_percentage_ewma'] = passer_df.groupby('passer_player_name')['completion_percentage']\
        .transform(lambda x: x.ewm(span=2, min_periods=1).mean())
    passer_df['pass_attempts_ewma'] = passer_df.groupby('passer_player_name')['pass_attempts']\
        .transform(lambda x: x.ewm(span=2, min_periods=1).mean())
    passer_df['air_yards_ewma'] = passer_df.groupby('passer_player_name')['air_yards']\
        .transform(lambda x: x.ewm(span=2, min_periods=1).mean())
    passer_df['yards_after_catch_ewma'] = passer_df.groupby('passer_player_name')['yards_after_catch']\
        .transform(lambda x: x.ewm(span=2, min_periods=1).mean())
    passer_df['epa_ewma'] = passer_df.groupby('passer_player_name')['epa']\
        .transform(lambda x: x.ewm(span=2, min_periods=1).mean())
    passer_df['interception_ewma'] = passer_df.groupby('passer_player_name')['interception']\
        .transform(lambda x: x.ewm(span=2, min_periods=1).mean())
    passer_df['qb_hit_ewma'] = passer_df.groupby('passer_player_name')['qb_hit']\
        .transform(lambda x: x.ewm(span=2, min_periods=1).mean())
    passer_df['sack_ewma'] = passer_df.groupby('passer_player_name')['sack']\
        .transform(lambda x: x.ewm(span=2, min_periods=1).mean())
    passer_df['pass_touchdown_ewma'] = passer_df.groupby('passer_player_name')['pass_touchdown']\
        .transform(lambda x: x.ewm(span=2, min_periods=1).mean())
    passer_df['passing_yards_ewma'] = passer_df.groupby('passer_player_name')['passing_yards']\
        .transform(lambda x: x.ewm(span=2, min_periods=1).mean())
    passer_df['cpoe_ewma'] = passer_df.groupby('passer_player_name')['cpoe']\
        .transform(lambda x: x.ewm(span=2, min_periods=1).mean())

    # Drop the non-EWMA columns after calculation
    passer_df = passer_df.drop(columns=['completion_percentage', 'air_yards', 'yards_after_catch', 'epa', 
                                        'interception', 'qb_hit', 'sack', 'pass_touchdown', 'cpoe'])

    return passer_df

In [184]:
def calculate_defense_ewma(defense_df):
    """
    Calculates EWMA for the defense dataframe.
    """
    defense_df['completion_percentage_ewma'] = defense_df.groupby('defteam')['completion_percentage']\
        .transform(lambda x: x.ewm(span=2, min_periods=1).mean())
    defense_df['pass_attempts_ewma'] = defense_df.groupby('defteam')['pass_attempts']\
        .transform(lambda x: x.ewm(span=2, min_periods=1).mean())
    defense_df['air_yards_ewma'] = defense_df.groupby('defteam')['air_yards']\
        .transform(lambda x: x.ewm(span=2, min_periods=1).mean())
    defense_df['yards_after_catch_ewma'] = defense_df.groupby('defteam')['yards_after_catch']\
        .transform(lambda x: x.ewm(span=2, min_periods=1).mean())
    defense_df['epa_ewma'] = defense_df.groupby('defteam')['epa']\
        .transform(lambda x: x.ewm(span=2, min_periods=1).mean())
    defense_df['interception_ewma'] = defense_df.groupby('defteam')['interception']\
        .transform(lambda x: x.ewm(span=2, min_periods=1).mean())
    defense_df['qb_hit_ewma'] = defense_df.groupby('defteam')['qb_hit']\
        .transform(lambda x: x.ewm(span=2, min_periods=1).mean())
    defense_df['sack_ewma'] = defense_df.groupby('defteam')['sack']\
        .transform(lambda x: x.ewm(span=2, min_periods=1).mean())
    defense_df['pass_touchdown_ewma'] = defense_df.groupby('defteam')['pass_touchdown']\
        .transform(lambda x: x.ewm(span=2, min_periods=1).mean())
    defense_df['passing_yards_ewma'] = defense_df.groupby('defteam')['passing_yards']\
        .transform(lambda x: x.ewm(span=2, min_periods=1).mean())
    defense_df['cpoe_ewma'] = defense_df.groupby('defteam')['cpoe']\
        .transform(lambda x: x.ewm(span=2, min_periods=1).mean())

    # Drop the non-EWMA columns after calculation
    defense_df = defense_df.drop(columns=['passing_yards', 'completion_percentage', 'air_yards', 'yards_after_catch', 
                                          'epa', 'interception', 'qb_hit', 'sack', 'pass_touchdown', 
                                          'pass_attempts', 'cpoe'])

    return defense_df

In [185]:
def apply_ewma_to_upcoming_week(passer_df, defense_df, schedules_df, upcoming_week):
    """
    Merges passer and defense EWMA data for the upcoming week.
    """
    # Step 1: Filter the schedules for the upcoming week
    week_games = schedules_df[schedules_df['week'] == upcoming_week]

    # Step 2: Merge passer and defense EWMA data
    full_with_ewma = passer_df.merge(defense_df, how='inner', on=['defteam', 'season', 'week', 'roof', 'surface'], suffixes=('_passer', '_defense'))

    # Step 3: Return the full dataframe with EWMA features for the upcoming week
    return full_with_ewma

In [186]:
# Example usage:
upcoming_week = get_upcoming_week(schedules_df)

# Filter passer and defense data to get only the last two weeks
passer_df_filtered = filter_most_recent_weeks(passer_df)
defense_df_filtered = filter_most_recent_weeks(defense_df)

# Calculate EWMA features for passer and defense
passer_df = calculate_passer_ewma(passer_df_filtered)
defense_df = calculate_defense_ewma(defense_df_filtered)

# Apply the EWMA features to the upcoming week
week_4_predictions = apply_ewma_to_upcoming_week(passer_df, defense_df, schedules_df, upcoming_week)

# Check the Week 4 predictions
week_4_predictions.head()

,game_id,passer_player_name,posteam,defteam,season,week,passing_yards,home_flag_passer,pass_attempts,roof,surface,completion_percentage_ewma_passer,pass_attempts_ewma_passer,air_yards_ewma_passer,yards_after_catch_ewma_passer,epa_ewma_passer,interception_ewma_passer,qb_hit_ewma_passer,sack_ewma_passer,pass_touchdown_ewma_passer,passing_yards_ewma_passer,cpoe_ewma_passer,home_flag_defense,completion_percentage_ewma_defense,pass_attempts_ewma_defense,air_yards_ewma_defense,yards_after_catch_ewma_defense,epa_ewma_defense,interception_ewma_defense,qb_hit_ewma_defense,sack_ewma_defense,pass_touchdown_ewma_defense,passing_yards_ewma_defense,cpoe_ewma_defense
0,2014_20_GB_SEA,A.Rodgers,GB,SEA,2014,20,178.0,False,32.0,outdoors,fieldturf,0.593750,32.0,283.00,57.00,-9.575883,2.00,6.0,1.00,1.0,178.00,-5.215820,True,0.638021,36.00,286.00,91.75,-2.919278,2.00,6.25,1.00,1.75,215.50,-1.829401
1,2016_20_GB_ATL,A.Rodgers,GB,ATL,2016,20,287.0,False,44.0,dome,fieldturf,0.608665,41.0,352.75,79.50,4.154047,1.25,7.5,1.75,2.5,259.75,-1.579102,True,0.633614,48.50,429.25,104.50,8.317508,1.00,8.25,2.75,2.75,331.75,0.719226
2,2023_20_TB_DET,B.Mayfield,TB,DET,2023,20,349.0,False,39.0,dome,fieldturf,0.666667,39.0,328.00,142.00,3.863917,2.00,7.0,4.00,3.0,349.00,-2.787055,True,0.666667,36.75,316.75,136.75,4.554088,1.75,6.50,3.50,2.50,328.50,-1.456032
3,2016_18_OAK_HOU,B.Osweiler,HOU,LV,2016,18,168.0,True,25.0,closed,fieldturf,0.575405,28.0,244.50,70.75,-4.467511,0.75,1.5,0.75,1.0,175.25,-9.594740,False,0.596471,27.25,234.00,69.75,2.785740,0.00,1.25,0.50,1.25,187.00,-5.526695
4,2023_22_SF_KC,B.Purdy,SF,KC,2023,22,255.0,False,38.0,dome,grass,0.605263,38.0,324.00,140.00,6.197958,0.00,9.0,1.00,1.0,255.00,-3.190586,True,0.639170,38.75,340.25,152.50,10.081359,0.00,8.00,1.25,1.75,283.00,1.441067
